<a href="https://www.kaggle.com/code/jatin2055/langchain-tools?scriptVersionId=261573374" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install langchain langchain_core langchain_openai langchain_community

In [5]:
from langchain_core.tools import tool

# Custom Tool

Langchain documentation: https://python.langchain.com/docs/integrations/tools/

# 1. Using @tool decorator

In [17]:
@tool
def concat_tool(a: list = ["Hello", " World", "!!!"]) -> str:
    """ 
        This tool is used to concat the values provided in the list.
        This will return back a String output.
    """

    
    output = "".join(str(val) for val in a)
    return output

In [18]:
# a tool acts as a Runnable
concat_tool.invoke({"a": ['abc', 123, 'def']})

'abc123def'

In [21]:
concat_tool.invoke({})

'Hello World!!!'

In [19]:
print(concat_tool.name)
print(concat_tool.description)
print(concat_tool.args)

concat_tool
This tool is used to concat the values provided in the list.
This will return back a String output.
{'a': {'default': ['Hello', ' World', '!!!'], 'items': {}, 'title': 'A', 'type': 'array'}}


In [24]:
print(concat_tool.args_schema.model_json_schema())

{'description': 'This tool is used to concat the values provided in the list.\nThis will return back a String output.', 'properties': {'a': {'default': ['Hello', ' World', '!!!'], 'items': {}, 'title': 'A', 'type': 'array'}}, 'title': 'concat_tool', 'type': 'object'}


# 2. Using Strcutured Tool

In [25]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [26]:
# creating tool's parameters schema
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="First Input to the Multiply tool")
    b: int = Field(required=True, description="Second Input to the Multiply tool")

In [28]:
def multiply_func(a: int, b: int) -> int:
    """
    Tool to perform multiplication of two numbers.
    """

    return a*b

In [30]:
mutiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "Muultiply",
    description = """ Tool to perform multiplication of two numbers a & b """,
    args_schema = MultiplyInput
)

In [31]:
print(mutiply_tool.name)
print(mutiply_tool.args)
print(mutiply_tool.description)

Muultiply
{'a': {'description': 'First Input to the Multiply tool', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second Input to the Multiply tool', 'required': True, 'title': 'B', 'type': 'integer'}}
Tool to perform multiplication of two numbers a & b


In [34]:
mutiply_tool.invoke({"a": 4, "b": 8})

32

# 3. Using BaseTool

In [53]:
from typing import Type
from langchain.tools import BaseTool

In [66]:
class MultiplyToolSchema(BaseModel):
    a: float = Field(required=True, description="First Input to the Multiply tool")
    b: float = Field(required=True, description="Second Input to the Multiply tool")

In [70]:
class MutiplyTool(BaseTool):
    name: str = "Multiplication Tool"
    description: str = "Tool to multiple two numbers"

    args_schema: Type[BaseModel] = MultiplyToolSchema

    # schema is verified only at args_schema level, but not at _run function
    # here even we can mention 'a' as str and the method will still run 
    def _run(self, a: str, b: int) -> float:
        print(f"a: {a}, type: {type(a)}")
        return a*b

In [71]:
multiply = MutiplyTool()

In [72]:
multiply.invoke({"a": 1.2, "b":"3.3"})

a: 1.2, type: <class 'float'>


3.9599999999999995

In [57]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

Multiplication Tool
Tool to multiple two numbers
{'a': {'description': 'First Input to the Multiply tool', 'required': True, 'title': 'A', 'type': 'number'}, 'b': {'description': 'Second Input to the Multiply tool', 'required': True, 'title': 'B', 'type': 'number'}}


# TOOLKIT

In [83]:
@tool
def fake_hello(a: str) -> str:
    """ Returns Hello """
    return 'Hello'


@tool
def fake_LangChain(b: str) -> str:
    """ Returns LangChain """
    return 'LangChain'

@tool
def concat_tool(c: list) -> str:
    """ Concats all the attributes in the array"""
    return "".join(str(val) for val in c)

In [84]:
class HelloWorldToolkit:
     def get_tools(self):
         return [fake_hello, fake_LangChain]

In [85]:
toolkit = HelloWorldToolkit()

tools = toolkit.get_tools()

In [96]:
# no other way to invoke a tool from the toolkit
tools[0].invoke({"a": 'Hi'})

'Hello'